## Introduction

As mentioned in the last notebook, it is necessary to inspect your data before analysis. For example, are you sure you know which file delimiter was used to create the file? Should the data be transposed? Data preprocessing steps also include inspecting the covariates and plotting data to search for outliers that may indicate measurement (or data entry) errors. 

We also discussed steps that can be used to determine whether the data fit classical statistical assumptions. This notebook continues in this area. In particular, we introduce generalize linear models (GLMs) that can be used to analyze non-normal data, such as count data or percentages.

For this notebook, we will use the glucosinolate data that we downloaded earlier.

### Load the glucosinolate data

In [ ]:
# rm(list=ls());
# open the glucosinolate file in R
# same file as before...
glucosinolateFileName <- "data/cmeyer_glucs2015/bmeyer_etal.txt";  
glucs <- read.table(glucosinolateFileName, header=T, sep="\t", as.is=T, stringsAsFactors=FALSE);  
glucs <- glucs[order(glucs[,"accession_id"]),];

In [ ]:
# what's in the working environment?
ls();

### What is the statistical model?

During today's notebook, we will discuss *Poisson* family analyses. Poisson models (and their derivatives) are used to analyze count data, which are very common.

Examples:
  - The number of microbial species in each of our gut microbiomes 
  - The number of people that die from hippo attacks each year
  - The number of reads assigned to a gene in an RNA-Seq dataset
 
The glucosinolate data are also count data.

In [ ]:
str(glucs)

In [ ]:
head(glucs);

In [ ]:
# if lme4, ggplot2, and gridExtra aren't installed, install them...
if( !require("lme4" )){  
    install.packages("lme4");  
}

if( !require("ggplot2" )){  
    install.packages("ggplot2");  
}

if( !require("gridExtra" )){  
    install.packages("gridExtra");  
}


### Replicates in the data

As noted earlier, the glucosinolate data were generated with *Arabidopsis thaliana*, the plant genetic model species. It is among the main model species used by plant geneticists in part because it is self compatible. This means that replicated inbred lines can be used in experiments to reduce experimental noise. Importantly, the mean phenotype of an inbred line can typically be measured with higher precision than for an outbred line.

### Estimate the mean phenotype per genotype

The speed of GWAS is affected by the sample size and the number of SNPs, as well as other factors. To speed up analyses, researchers working with repeated measures data (e.g. longitudinal analyses) or inbred lines often reduce the number of observations for each sample or individual. One approach, as illustrated in the previous notebook, is to simply average the phenotypic data per individual. However, this is not ideal (i.e. the errors propagate when analyzing the "means of means").

In [ ]:
# recall, there is more than one observation per inbred line
counts <- table(glucs$accession_id);
cat("The distribution of the number of replicates per accession\n");
table(counts);

# earlier, we visualized this as a barplot:
# ggplot() + aes(counts) + 
#         xlab( paste0( "The number of replicates" )) + ylab("Counts") +
#         geom_bar(stat="count", fill="tan1") + 
#         geom_vline(aes(xintercept=mean(counts)), linetype=3);

But how should we extract the mean phenotype per individual?

Recall that, earlier, we used linear mixed models to fit BLUPs:

In [ ]:
# previously, we used a mixed model to specify a random effect with the following code:
lmer0 <- lmer( G2P ~ 1 + (1|accession_id), data=glucs );
linearBlups <- ranef(lmer0)$accession_id; # these are the best-linear unbiased predictors:
linearBlups <- data.frame( accession_id=rownames(linearBlups), linear_blup=linearBlups[,1], stringsAsFactors=FALSE );
head(linearBlups);

This is a standard approach with mixed models. However, you'll remember that these are not really *normally distributed* data.

Indeed, a lot of the data that biologists work with are non-normal (e.g.):
  - number of offspring 
  - infection rates
  - survival/death
  - gene expression data
  - metabolomic data

In general, it is appropriate to analyze such data with generalized linear models (or GLMs). In the case of *count* data, it is typical to use *Poisson* (or Poisson-family) models. An easy rule-of-thumb to remember is that if the data are all non-negative integers, with no natural upper bound, then the Poisson-family models should be used.


<!--# however, these are count data (see the output from head above), which suggests 
# that a Poisson-family model should be used; let's try a simple quasi-Poisson GLM:
glm0 <- glm( G2P ~ 1, data=glucs, family="quasipoisson" );
glm0.res <- residuals( glm0 );
glm0.res[1:10];

# note: the names are no longer the accession Ids, but the row names from the glucs data frame. Do you know why?
# let's use a workaround:
################################################################################
## my version of stack, which avoids factor generation
################################################################################
mstack <- function(arg, newHeaders, setRowNames=T, sorted=TRUE, decreasing=F){
    values <- data.frame(names=I(names(arg)), values=as.numeric(arg));

    if( setRowNames ){
        rownames(values) <- values[,"names"];
    }
    
    if( sorted ) {
        values <- values[order(values[,"values"], decreasing=decreasing),];	
    }
    
    colnames(values) <- newHeaders;
    return(values);
}

glm0.res <- mstack( glm0.res, newHeaders=c("row_id", "residual"), sorted=FALSE );
head(glm0.res);-->

In [ ]:
# the estimates from Meyer et al., however, are areas estimated under the curve, from a QQQ Mass spectrometer.
# these can be rounded without a loss of precision 
glucs[,3:ncol(glucs)] <- round(glucs[,3:ncol(glucs)]);
glucs[1:5,];

In [ ]:
# now, these integers can be fit with GLMs/GLMMs:
glmer0 <- glmer( G2P ~ 1 + (1|accession_id), data=glucs, family="poisson" );
glmBlups <- ranef(glmer0)$accession_id; # these are the best-linear unbiased predictors:
glmBlups <- data.frame( accession_id=rownames(glmBlups), pois_blup=glmBlups[,1], stringsAsFactors=FALSE );
head(glmBlups);

For poisson/negative binomial GLMs the distribution of these random effects (or residuals, if a GLM is used) will tend to be skewed, which is to be expected. For larger counts (and larger $\lambda$) the residuals will tend to normality.

### Poisson-family models allow offsets

When the counts from a Poisson process should be modeled as a rate, then an offset should be included in the model. Specifying an exposure will thus enable you to express the count data as a function of the *effort* that was necessary to collect the data. For example, if you detect 25,000 counts for G2P in a sample with a high background on a mass spec (the instrument that Meyer et al., used to measure glucosinolates), then you should take that higher *ion count* into **account** when comparing that sample with samples that have less of a background. 

As another example, if two botanists count the number of species at the botanical gardens and one botanist spends 24 hours counting species and the other 72 hours (assuming the same skill level), then you would typically expect the second botanist to count **more species**, given the extra effort.

But how do you specify an offset?
<!--log𝜇𝑥 = $\beta$0 + $\beta$1𝑥 

(where 𝜇𝑥 is the expected count for those with covariate 𝑥), you have

log𝜇𝑥𝑡𝑥=𝛽′0+𝛽′1𝑥
(where 𝑡𝑥 is the exposure time for those with covariate 𝑥). Now, the last equation could be rewritten

log𝜇𝑥=log𝑡𝑥+𝛽′0+𝛽′1𝑥
and log𝑡𝑥 plays the role of an offset.-->

In [ ]:
# determine the total ion count
totalIonCounts <- rowSums( glucs[,-c(1,2)] );
head(totalIonCounts);

In [ ]:
# now include the total ion count as an offset
# if neither df has been sorted, you can immediately add it to the model without merging with the original df
glm0 <- glm( G2P ~ 1, offset(log(totalIonCounts)), data=glucs, family="quasipoisson");

Wait a minute, negative weight? The only weighting comes from the offset, which is expressed as a log in Poisson-family models.

In [ ]:
which(log(totalIonCounts) < 0);
which(totalIonCounts == 0);

### Quality control (data curation) is an ongoing process

There are samples in the data that have *no* ion counts. If this wasn't a simple mistake during data collection (e.g. failing to input the data into the file), then the samples could be rerun or dropped. We have a lot of data, and you're a busy person, so let's drop them.

<!-- # glm0 <- glm( G2P ~ 1, offset(log(totalIonCounts)), data=glucs2, family="quasipoisson");
# summary(glm0);-->

In [ ]:
dropouts <- which( totalIonCounts == 0 );
glucs2 <- glucs[-dropouts,];
dim(glucs2);

totalIonCounts <- rowSums( glucs2[,-c(1,2)] );
cat("The range of ion counts is now:", range(totalIonCounts), "\n");

In [ ]:
# 20 is a small number, but the advantage of adding an offset is 
# that you can include as much data as possible, and let the model weight things appropriately

glmer0 <- glmer( G2P ~ 1 + offset(log(totalIonCounts)) + (1|accession_id), data=glucs2, control=glmerControl(optimizer="bobyqa", optCtrl=list(maxfun=2e5)), family="poisson" );
glmBlups <- ranef(glmer0)$accession_id; # these are the best-linear unbiased predictors:
glmBlups <- data.frame( accession_id=rownames(glmBlups), pois_blup=glmBlups[,1], stringsAsFactors=FALSE );
tail(glmBlups);

These BLUPs can be used in GWAS.